# Deep learning technical model for ETFs

Based on the kibot data

In [1]:
# preamble
%alias_magic  nbagg matplotlib -p nbagg
%alias_magic  inline matplotlib -p inline
%load_ext memory_profiler

from io import BytesIO
from IPython.display import display, Image, HTML
from pylab import *
plt.style.use( 'seaborn-whitegrid' )

from madmax.api import *
mx = mx.reload()
mxtr = mxtr.reload()

from research import yahoo; reload( yahoo)


###################################################################
# Interactive configuration 
# When running a config grid this will throw a FrozenException 
# and the grid Config will be used
###################################################################
try:
    mx.Config.name = 'etf_technical/deep_learning/2020.04.17'
    mx.Config.mode = 'interactive'
    mx.Config.hps = mx.HPSet(
        # trade during market hours     
        market_hours=True,
        # trade after market hours
        pre_market=False,
        # trade pre market hours 
        post_market=False,
    )
    mx.Config.code = 'etf_deep_learning_2020.04.17.ipynb'
    disp( 'Running template configuration ', h=2 )        
except mx.Config.FrozenException:
    if mx.Config.mode != 'grid':
        raise RuntimeError( 'An unexpected configruation encountered' )
    disp( 'Running a grid configuration ', h=2 )

hps = mx.Config.hps
mx.Config.start()
    

Created `%nbagg` as an alias for `%matplotlib nbagg`.
Created `%inline` as an alias for `%matplotlib inline`.


<IPython.core.display.Javascript object>

<h1> Config etf_technical/deep_learning/2020.04.17 </h1>
<b> hash</b>: none </h4>
<b> mode </b>: interactive
<b> frozen </b>: False
<b> uri </b>: /home/fj/experiments/etf_technical/deep_learning/2020.04.17/none
<h3> Hyperparameters </h3>
HPSet(
		market_hours=True,
		post_market=False,
		pre_market=False,
	)<h3> Code </h3>
<b> format </b>: .ipynb
<b> code </b> : /local/cloud/code/research/etf_technical/etf_deep_learning_2020.04.17.ipynb

In [2]:
# the various features in the data
####################################################################
from research.etf_technical import load_data as eld; reload( eld )


price_features = ['open_30min', 'high_30min', 'low_30min', 'close_30min', 
                  'open_day', 'high_day', 'low_day', 'close_day', 'close_prev_day', 
                  'open_week', 'high_week', 'low_week', 'close_week', 'close_prev_week', 
                  'open_month', 'high_month', 'low_month', 'close_month', ]
                      
volume_features = ['volume_30min', 'volume_intraday', 'volume_intraweek', 'volume_1hr', 'volume_1day', 
                       'dollar_volume_30min', 'dollar_volume_intraday',
                       'dollar_volume_intraweek', 'dollar_volume_1hr', 'dollar_volume_1day',]
                       
calendar_features = [ 'week', 'month', 'timeofday', 'dayofweek', 'dayofmonth', 
                          'weekofyear', 'cos_timeofday', 'sin_timeofday', 
                          'cos_dayofweek', 'sin_dayofweek', 'cos_dayofmonth',
                          'sin_dayofmonth', 'cos_weekofyear', 'sin_weekofyear',]

returns_features = ['logrtn_lag_intraday', 'logrtn_lag_overnight', 'logrtn_lag_intraweek',
                    'logrtn_lag_weekend', 'logrtn_lag_intramonth', 'logrtn_lag_30min',
                    'logrtn_lag_1hr','logrtn_lag_1day', ]

responses = ['logrtn_lead_30min',  'logrtn_lead_1hr', 'logrtn_lead_1day'] #'logrtn_lead_intraday', 



################################################################################################
# build features for the deep learning (same as teh code for the 30-min online model)
################################################################################################
@mx.operatorize( memoize='md', consumes_features=False, produces_meta=True )
def FeatureBuilder( data, responses=responses,  emv=True, ema=True, equalize_lagging_returns=False,
                    haar=False, dilation=2, levels=5, winsorize=0.05, smz=False,
                    volume_features=volume_features, returns_features=returns_features,
                    verbose=False ):
    '''
    build up the 30 min dataset for learning
    
    equalize_lagging_returns:
        divide out annual vol from all lagging returns features
        Leading returns features are equalized anyways
    '''
    INTS_PER_DAY = 13
    logger = mx.Logger( 'build_features', verbose=verbose )
    das = []
    with logger.timer( 'loading the data'):
        for symbol in logger.pbar( eld.kibot_day_files.keys() ):
            if symbol.startswith( 'INCOMPLETE' ):
                logger.warn( f'No data for {symbol} - SKIPPING' )
                continue
            logger.info( 'loading data for ', symbol )
            da, meta = eld.build_30min_data( symbol=symbol )
            das.append( da )
        da = xa.concat( das, dim='symbol' )
    features = volume_features + returns_features
    with logger.timer( ' compute log features' ):
        for ft in ['dollar_volume_30min', 'open_30min', 'dollar_volume_intraday',  'dollar_volume_intraweek',] :
            da = da.assign_features( **{ f'log_{ft}': da.loc[:,:, ft].log() } )
            features.append( f'log_{ft}' )
    with logger.timer(' compute an annual returns volatility'):
        avc = mx.transforms.rolling( windows='252D', sid='symbol', min_periods=INTS_PER_DAY*60 ).sd()
        da = avc( da, features='logrtn_lag_30min' )
        annual_vol = avc.output_features[0]
        # equalize the response variable
        equalized_responses = []
        for response in responses:
            equalized_response = f'{response}_equalized'
            da = da.assign_features( **{equalized_response: da.loc[:,:,response] / ( da.loc[:,:,annual_vol] + 1e-12 ) } )
            equalized_responses.append( equalized_response )
    if equalize_lagging_returns:
        # equalize all the laggign returns columns inplace ....
        with logger.timer( 'compute "normalized" leading and lagging returns' ):
            da.loc[:,:,returns_features] =  da.loc[:,:,returns_features] / ( da.loc[:,:,annual_vol] + 1e-12 )
            # volume, price and returns vol
    if emv:
        with logger.timer( 'compute volatility' ):
            emv = mx.transforms.exponential( halflifes=np.array( [2, 10, 30] ) * INTS_PER_DAY, sid='symbol' ).sd()
            da = emv( da, features=['log_open_30min', 'log_dollar_volume_30min', 'logrtn_lag_30min', 'logrtn_lag_1day'] )
            features += emv.output_features
    # volume, price and returns sma
    if ema:
        with logger.timer( 'compute ema' ):
            ema = mx.transforms.exponential( halflifes=np.array( [2, 10, 30] ) * INTS_PER_DAY, sid='symbol' ).mean()
            da = ema( da, features=['log_open_30min', 'log_dollar_volume_30min', 'logrtn_lag_30min', 'logrtn_lag_1day'] )
            features += ema.output_features
            # compute haar transform
    if haar:
        with logger.timer( 'compute haar' ):
            haar = mx.transforms.haar( levels=levels, dilation=dilation, sid='symbol' )
            haar.features = ['log_open_30min', 'log_dollar_volume_30min', 'logrtn_lag_30min', 'logrtn_lag_1day']
            da = haar( da )
            features += haar.output_features
    # winsorize the features we've constructed and other good stuff
    if winsorize:
        with logger.timer( 'winsorizing' ):
            wins = mx.transforms.batch( sid='symbol' ).winsorize( quantiles=winsorize )
            wins.features = sorted( set(features + [annual_vol] + equalized_responses  ) )
            da = wins( da )
            # drop the original features and rename the winsorized one
            da = (da.drop_coords( features=wins.features )
                  .rename_coords( features=dict( zip( wins.output_features, wins.features ) ) ))
    # zscore the features
    if smz:
        with logger.timer( 'zscoring' ):
            zscore = mx.transforms.rolling( windows=smz, sid='symbol' ).zscore()
            da = zscore( da, features=features )
            # drop the original features and rename the zscored one
            da = da.drop_coords( features=zscore.features ).rename_coords( features=dict( zip( zscore.output_features, zscore.features ) ) )
    # add a dummy weight column
    da = da.assign_features( one=1 )
    return da, features, dict( equalized_responses=equalized_responses, annual_vol=annual_vol )


In [3]:
# --- build data nad features ---
######################################################################################################
builder = FeatureBuilder( responses=responses, volume_features=volume_features, returns_features=returns_features, smz=False  )
builder.data = None

da = builder()
disp( 'expected got', builder.hash() )

# obtain the metadata for each symbol
metas = []
for symbol in da.symbol.values:
    for etf_list in [eld.bond_etfs, eld.equity_etfs, eld.vol_etfs]:
        if symbol in etf_list.Symbol.values:
            metas.append( etf_list[etf_list.Symbol == symbol].iloc[0].to_dict() )
            break

meta_df =pd.DataFrame(metas)     

/home/fj/cloud/code/madmax/experiment/logging.py:104: UserWarning: Logger.log_uri not set. Try running Config.start() ? Will not write to disk 
  warnings.warn( 'Logger.log_uri not set. Try running Config.start() ? Will not write to disk ' )


## Setup learning hyperparameters

In [4]:
# -- learning hyperparams ---

mxtr = mxtr.reload() 
hps = mx.HPSet()
hps.features =[ 
    'logrtn_lag_overnight', 'logrtn_lag_weekend', 'logrtn_lag_30min', 'logrtn_lag_1hr', 'logrtn_lag_1day',
    'logrtn_lag_intraday', 'logrtn_lag_intraweek',    'logrtn_lag_intramonth',
    'log_open_30min', 'log_dollar_volume_30min', 'log_dollar_volume_intraday', 'log_dollar_volume_intraweek',] #
#      'log_open_30min_26.ew_sd',  'log_dollar_volume_30min_26.ew_sd',  'logrtn_lag_30min_26.ew_sd',  'logrtn_lag_1day_26.ew_sd',
#      'log_open_30min_130.ew_sd', 'log_dollar_volume_30min_130.ew_sd', 'logrtn_lag_30min_130.ew_sd', 'logrtn_lag_1day_130.ew_sd',
#      'log_open_30min_390.ew_sd', 'log_dollar_volume_30min_390.ew_sd', 'logrtn_lag_30min_390.ew_sd', 'logrtn_lag_1day_390.ew_sd',
#      'log_open_30min_26.ew_mean', 'log_dollar_volume_30min_26.ew_mean', 'logrtn_lag_30min_26.ew_mean', 'logrtn_lag_1day_26.ew_mean',
#      'log_open_30min_130.ew_mean', 'log_dollar_volume_30min_130.ew_mean', 'logrtn_lag_30min_130.ew_mean', 'logrtn_lag_1day_130.ew_mean',
#      'log_open_30min_390.ew_mean', 'log_dollar_volume_30min_390.ew_mean', 'logrtn_lag_30min_390.ew_mean', 'logrtn_lag_1day_390.ew_mean'    
hps.calendar_features = ['cos_timeofday', 'sin_timeofday', 'cos_dayofweek', 'sin_dayofweek', 
                         'cos_dayofmonth','sin_dayofmonth', 'cos_weekofyear', 'sin_weekofyear']
hps.responses = [ 'logrtn_lead_1day'] #['logrtn_lead_30min_equalized', 'logrtn_lead_1hr_equalized', 'logrtn_lead_1day_equalized'] #
hps.weight = 'one'
hps.lookhead = 14 # 1 day lookahead

device = 'cuda'
dtype = tr.float32

## Setup the dataset for learning


In [5]:
#setup dataset
#-----
%nbagg
hps.batch_size = 13 * 256 * 1 # yrs worth of batches
hps.priming = 13 * 3 * (2 ** 4)  #  priming when computing the loss is related to the dilation and num resnetl laysers
hps.data_standardize = False

T_da = da.time.astype( np.int64 ) / mx.TIME_TO_INT
X_da = da.loc[:,:,hps.features + hps.calendar_features]
#C_da = da.loc[:,:,hps.calendar_features]
Y_da = da.loc[:,:,hps.responses]
W_da = da.loc[:,:,hps.responses].isfinite()

if hps.data_standardize:
    X_da = (X_da / X_da.std( dim=['time', 'symbol'] ) )
    Y_da = (Y_da / Y_da.std( dim=['time', 'symbol'] ) )
X_da = X_da.fillna( 0 )
Y_da = Y_da.fillna( 0 )

# build the mask generator
masker = mxtr.PanelMasker( 
    data=da, priming=hps.priming, num_its=8,
    tst_split=mx.SplitData.time_cutoff, tst_frac=0.25,
    val_split=mx.SplitData.weekly_random, val_frac=0.25,
    trg_mask_size=1000, val_mask_size=None, tst_mask_size=None,    
)
masker.restrict( da )    
trg_mask= masker.mask( 'trg' )
val_mask= masker.mask( 'val' )
tst_mask = masker.mask( 'tst' )
assert len(set(trg_mask.values).union( set( tst_mask.values)).union( set(val_mask.values))) == len(da) - hps.priming
disp( 'trg_frac=', len(trg_mask) / len(da), 'val_frac=', len(val_mask) / len(da), 'tst_frac=', len(tst_mask) / len(da) )

full_dataset = mxtr.TensorDataset(     
    T_da.to_tensor(  ), # do not change this dtype - otherwise you lose precision
    X_da.to_tensor( dtype=dtype ),     
    Y_da.to_tensor( dtype=dtype ),  
    W_da.to_tensor( dtype=dtype ),  
) 

# the training + validation dataset
trg_dataset = mxtr.TensorDataset(     
    T_da[:tst_mask.min()].to_tensor(  ), 
    X_da[:tst_mask.min()].to_tensor( dtype=dtype ),     
    Y_da[:tst_mask.min()].to_tensor( dtype=dtype ),  
    W_da[:tst_mask.min()].to_tensor( dtype=dtype ),  
)
tst_dataset  = mxtr.TensorDataset(     
    T_da[tst_mask].to_tensor(  ), 
    X_da[tst_mask].to_tensor( dtype=dtype ),     
    Y_da[tst_mask].to_tensor( dtype=dtype ),  
    W_da[tst_mask].to_tensor( dtype=dtype ),  
)

# build up the dataloader object
sampler = mxtr.PanelSampler( trg_dataset, batch_size=hps.batch_size )
dataloader = tr.utils.data.DataLoader( trg_dataset, batch_sampler=sampler, pin_memory=True )
# for i, mb in enumerate( dataloader ):
#     print( i , pd.to_datetime( mb[0].values * mx.TIME_TO_INT ).min() )

if False:
    Y_da.std( dim=['time']).to_dataframe_mx().hist(bins=100)

    figure(figsize=(20,20))
    for i, c in enumerate( X_da.features.values ):
        ax = subplot( 5, 5, i+1 )
        X_da.loc[:,:,c].std( dim=['time']).to_series().hist(bins=100, ax=ax )
        ax.set_title( c )


## The model 

In [6]:
# -- setup the model --
mxtr = mxtr.reload()
from research.etf_technical import model_v1 as mdl; reload(mdl)
mx.Config.start()

disp( mx.tensorboard.start( clear=True ) )
disp( mx.tensorboard.start_server() )

#learner = mxtr.Learner( optimizers=[],)    

hps.chi2 = None
hps.curvature = None
hps.dirichlet = None
hps.lasso = None
hps.lipschitz = None
hps.logdet = None
hps.neumann = None
hps.ridge = 1e-9
hps.smooth_lasso = None
hps.tv = 1e-9
hps.error =  mxtr.Error( metric='l2' )
hps.activation = 'leaky_relu'
    
mx.seed( 17 )
model = mdl.FactorResnet( 
    num_features = len(hps.features+hps.calendar_features),
    num_responses=len( hps.responses ),
    L=13 * 3, 
    dropout=False,
    batch_norm=False,
    resnet_layers=3,
    resnet_gating=False,
    resnet_channels=10,
    resnet_activation=hps.activation,
    dilation=2,
    monitor=True
).initialize( 'normal' ).to( dtype ).to( device )

disp( model.describe(), size=5 )

def trg_loss( times, X, Y, W, mask=None, patience=1  ):
    ''' the training loss composed of error + regularizations '''
    # build up the regularization terms
    model.train( True )
    act_reg, Yhat = model.activation_regularization( X, chi2=hps.chi2, tv=hps.tv, curvature=hps.curvature, dirichlet=hps.dirichlet, neumann=hps.neumann  )
    weight_reg = model.weight_regularization( ridge=hps.ridge, lasso=hps.lasso, smooth_lasso=hps.smooth_lasso, logdet=hps.logdet  )    
    lip_reg = (model.lipchitz_regularization( X=X, penalty=hps.lipschitz, lr=0.5, radius=1e-2, num_its=5, tensorboard=True  )[0] if hps.lipschitz else 0)
    reg = act_reg + weight_reg + lip_reg
    # the error term
    err = hps.error( Yhat=Yhat, Y=Y, W=W, mask=mask )
    # tensorboard logging
    if not hasattr( trg_loss, 'it_counter' ):
        trg_loss.it_counter = 0
    if trg_loss.it_counter % patience == 0:
        Wsum = W[mask].sum() if mask is not None else W.sum()
        myerr = (Yhat - Y).sq() 
        myerr = myerr[mask].sum() if mask is not None else myerr.sum()
        try:
            mx.tensorboard.logger.add_scalar( f'TrgLoss/error', err, trg_loss.it_counter  )        
            mx.tensorboard.logger.add_scalar( f'TrgLoss/reg/total', reg, trg_loss.it_counter )        
            mx.tensorboard.logger.add_scalar( f'TrgLoss/reg/weight', weight_reg, trg_loss.it_counter )        
            mx.tensorboard.logger.add_scalar( f'TrgLoss/reg/activation', act_reg, trg_loss.it_counter )        
            mx.tensorboard.logger.add_scalar( f'TrgLoss/reg/lipschitz', lip_reg, trg_loss.it_counter )        
            mx.tensorboard.logger.add_scalar( f'TrgLoss/minibatch_start', min( times.values ), trg_loss.it_counter  )            
            #mx.tensorboard.logger.add_scalar( f'TrgLoss/Wsum', Wsum, trg_loss.it_counter  )                        
            #mx.tensorboard.logger.add_scalar( f'TrgLoss/unweighted_err', myerr, trg_loss.it_counter  )                        
            mx.tensorboard.logger.add_text( f'TrgLoss/minibatch_start', str( pd.to_datetime( min( times.values ) * mx.TIME_TO_INT ) ) , trg_loss.it_counter  )            
        except Exception as e:
            warnings.warn( f'tensorboard failure {e} \n SKIPPING' )
    trg_loss.it_counter += 1
    return err + reg

def val_loss( times, X, Y, W, mask=None, patience=10 ):
    model.train( False )
    Yhat = model( X )
    err = mxtr.Error( 'gsr' )( Yhat, Y, W, mask=mask ).item() * np.sqrt( 252 )
#     if not hasattr( val_loss, 'it_counter' ):
#         val_loss.it_counter = 0
#     if val_loss.it_counter % patience == 0:
#         with tr.enable_grad():
#             lip_reg, _ = model.lipchitz_regularization( X, penalty=1, tensorboard=True, num_its=50 )
#         mx.tensorboard.logger.add_scalar( f'ValLoss/error', err, val_loss.it_counter  )        
#         mx.tensorboard.logger.add_scalar( f'ValLoss/lip_reg', lip_reg, val_loss.it_counter  )        
#     val_loss.it_counter += 1
    return err


/home/fj/cloud/code/madmax/pytorch/module.py:176: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat( res ).set_index( ['name', 'type', 'shape', 'numel', 'norm', 'dtype', 'device'] ).reset_index()


,name,type,shape,numel,norm,dtype,device,arguments
0,,FactorResnet,,19551,14.211843,,,"{'num_features': 20, 'num_responses': 1, 'L': ..."
1,first_layer/,OneLayer,,7810,3.185195,,,"{'layer': 'first', 'L': 39, 'Pin': 20, 'Pout':..."
2,first_layer/convolution/,CausalConv1D,,7810,3.185195,,,"{'L': 39, 'Pin': 20, 'Pout': 10, 'stride': 1, ..."
3,first_layer/convolution/kernel,Parameter,"(39, 20, 10)",7800,3.113180,torch.float32,cuda:0,
4,first_layer/convolution/bias,Parameter,"(10,)",10,0.072014,torch.float32,cuda:0,
5,first_layer/activation/,LeakyReLU,,0,0.000000,,,"{'negative_slope': 0.01, 'inplace': False}"
6,first_layer/monitor/,FlowMonitor,,0,0.000000,,,"{'name': 'FlowMonitor/first', 'patience': 10}"
7,resnet/,ResNet,,11730,9.784176,,,"{'layers': [OneLayer( layer='resnet_0', ..."
8,resnet/layers/,ModuleList,,11730,9.784176,,,{'modules': <bound method Module.modules of Mo...
9,resnet/layers/0/,OneLayer,,3910,3.258266,,,"{'layer': 'resnet_0', 'L': 39, 'Pin': 10, 'Pou..."


In [12]:
# -setup the learner ---
############################
mxtr = mxtr.reload()
from research.etf_technical import model_v1 as mdl; reload(mdl)
mx.Config.start()

disp( mx.tensorboard.start( clear=True ) )
disp( mx.tensorboard.start_server() )

opt = mxtr.RAdam( params=model.parameters(), lr=1e-3, betas=( 0.9, 0.99 ), tensorboard=True ) # mx.Resolve.halflife( 10)['decay'], mx.Resolve.halflife( 20 )['decay'] ) )
# lr_scheduler = mxtr.CycleAdamLR( optimizer=opt, min_lr=1e-5, max_lr=1e-4, min_betas=(0.6,0.8), max_betas=(0.9, 0.9), T_max=200 )
# lr_scheduler = mxtr.WarmupAdamLR( optimizer=opt, min_lr=1e-4, max_lr=1e-3, min_betas=(0.5,0.6), max_betas=(0.9, 0.9), T_max=500 )
learner = mxtr.Learner(
    optimizers=opt,
    trg_loss=trg_loss,
    val_loss=val_loss,    
    max_its=5000,
    device=device,
    callbacks=[
        mxtr.callbacks.LossLogger( patience=1 ),
        mxtr.callbacks.ParameterLogger( named_params=model.named_parameters(), patience=10 ),
        mxtr.callbacks.ParameterMacd( hl_fast=10, hl_slow=100, patience=5, rewind_factor=None  ),
        #mxtr.callbacks.ModelSnapshots( model=model, period=100, on_minimum=True, val_halflife=10 )
        #mxtr.calllbacks.ClipGradients( signize=False, clip=None, quantile=None, winlen=100 ),
#         mxtr.callbacks.LRSchedule( lr_scheduler ),
    ]    
)
disp( 'learner hash ', learner.hash(), h=4)
disp( hps )

In [13]:
# --- do the fitting ---
# for it in pbar( learner.fit_iterator( data=trg_dataset[20000:30000], masker=None, val_data=trg_dataset[35000:40000] ) ):
#for it in pbar( learner.fit_iterator( data=trg_dataset[-20000:-5000], masker=None, val_data=tst_dataset ) ):
#for it in pbar( learner.fit_iterator( data=trg_dataset[30000:40000], masker=None, val_data=trg_dataset[45000:50000] ) ):
for it in pbar( learner.fit_iterator( data=trg_dataset[20000:30000], masker=None, val_data=trg_dataset[35000:40000] ) ):
    if it % 100 == 0:
        print( model.hash() )
        disp( learner.profiles.sort_values( 'duration_mean', ascending=False ) )

/home/fj/cloud/code/madmax/pytorch/optim/callbacks.py:432: RuntimeWarning: invalid value encountered in float_scalars
  / np.sqrt( (curr_grad ** 2).mean() * (prev_grad ** 2).mean() ) + 1e-64)


FactorResnet_0085382bf


,num_calls,duration_total,duration_mean,duration_squared,duration_std
next( trg_iterator ),100,78.514244,7.773688e-01,1.512623e+00,9.530586e-01
next( val_iterator ),100,58.297054,5.771986e-01,1.405783e+00,1.035676e+00
_build_val_iterator:::self.val_loss,100,58.268517,5.769160e-01,1.405714e+00,1.035800e+00
_build_trg_iterator::loss.backward(),100,35.065578,3.471839e-01,1.194228e+00,1.036191e+00
_build_trg_iterator::self.trg_loss,100,13.628188,1.349326e-01,1.847607e-02,1.640968e-02
trg::self._resolve_data,1,0.054808,2.740419e-02,1.501979e-03,2.740419e-02
val::self._resolve_data,1,0.026300,1.314986e-02,3.458375e-04,1.314986e-02
ParameterLogger.post_iteration,100,0.721266,7.141250e-03,3.721584e-04,1.792096e-02
optimizer step,100,0.218548,2.163842e-03,4.775850e-06,3.060025e-04
ParameterMacd.post_iteration,100,0.049781,4.928844e-04,5.567821e-07,5.602205e-04


FactorResnet_82fb21aed


,num_calls,duration_total,duration_mean,duration_squared,duration_std
next( trg_iterator ),200,157.058469,7.813854e-01,1.517827e+00,9.525042e-01
next( val_iterator ),200,116.706347,5.806286e-01,1.416111e+00,1.038740e+00
_build_val_iterator:::self.val_loss,200,116.675626,5.804757e-01,1.416061e+00,1.038801e+00
_build_trg_iterator::loss.backward(),200,70.014458,3.483306e-01,1.197186e+00,1.037233e+00
_build_trg_iterator::self.trg_loss,200,27.277038,1.357067e-01,1.859369e-02,1.331874e-02
trg::self._resolve_data,1,0.054808,2.740419e-02,1.501979e-03,2.740419e-02
val::self._resolve_data,1,0.026300,1.314986e-02,3.458375e-04,1.314986e-02
ParameterLogger.post_iteration,200,1.399153,6.960960e-03,3.469758e-04,1.727776e-02
optimizer step,200,0.434616,2.162269e-03,4.727980e-06,2.292862e-04
ParameterMacd.post_iteration,200,0.100493,4.999673e-04,4.246156e-07,4.179094e-04


...

,num_calls,duration_total,duration_mean,duration_squared,duration_std
next( trg_iterator ),4700,3703.614785,7.878355e-01,1.538768e+00,9.581666e-01
next( val_iterator ),4700,2748.877948,5.847432e-01,1.429882e+00,1.043052e+00
_build_val_iterator:::self.val_loss,4700,2748.746550,5.847153e-01,1.429850e+00,1.043052e+00
_build_trg_iterator::loss.backward(),4700,1654.563536,3.519599e-01,1.214578e+00,1.044367e+00
_build_trg_iterator::self.trg_loss,4700,642.686982,1.367128e-01,1.877782e-02,9.350067e-03
trg::self._resolve_data,1,0.054808,2.740419e-02,1.501979e-03,2.740419e-02
val::self._resolve_data,1,0.026300,1.314986e-02,3.458375e-04,1.314986e-02
ParameterLogger.post_iteration,4700,32.942054,7.007457e-03,3.502563e-04,1.735373e-02
optimizer step,4700,10.361404,2.204085e-03,4.900867e-06,2.070637e-04
LossLogger.post_iteration,4700,2.308195,4.910009e-04,2.450095e-07,6.267027e-05


FactorResnet_26ae441a6


,num_calls,duration_total,duration_mean,duration_squared,duration_std
next( trg_iterator ),4800,3782.607835,7.878792e-01,1.539070e+00,9.582881e-01
next( val_iterator ),4800,2807.786703,5.848337e-01,1.430339e+00,1.043220e+00
_build_val_iterator:::self.val_loss,4800,2807.653052,5.848059e-01,1.430306e+00,1.043220e+00
_build_trg_iterator::loss.backward(),4800,1689.953576,3.520003e-01,1.214862e+00,1.044489e+00
_build_trg_iterator::self.trg_loss,4800,656.368007,1.367149e-01,1.877814e-02,9.337714e-03
trg::self._resolve_data,1,0.054808,2.740419e-02,1.501979e-03,2.740419e-02
val::self._resolve_data,1,0.026300,1.314986e-02,3.458375e-04,1.314986e-02
ParameterLogger.post_iteration,4800,33.659353,7.010905e-03,3.506394e-04,1.736337e-02
optimizer step,4800,10.584274,2.204598e-03,4.903319e-06,2.075264e-04
LossLogger.post_iteration,4800,2.357666,4.910782e-04,2.450588e-07,6.245752e-05


FactorResnet_f10f165b3


,num_calls,duration_total,duration_mean,duration_squared,duration_std
next( trg_iterator ),4900,3861.642904,7.879296e-01,1.539403e+00,9.584204e-01
next( val_iterator ),4900,2866.514332,5.848836e-01,1.430574e+00,1.043305e+00
_build_val_iterator:::self.val_loss,4900,2866.378400,5.848558e-01,1.430542e+00,1.043305e+00
_build_trg_iterator::loss.backward(),4900,1725.347343,3.520399e-01,1.215173e+00,1.044625e+00
_build_trg_iterator::self.trg_loss,4900,670.049885,1.367170e-01,1.877857e-02,9.329325e-03
trg::self._resolve_data,1,0.054808,2.740419e-02,1.501979e-03,2.740419e-02
val::self._resolve_data,1,0.026300,1.314986e-02,3.458375e-04,1.314986e-02
ParameterLogger.post_iteration,4900,34.349600,7.008692e-03,3.504061e-04,1.735754e-02
optimizer step,4900,10.804152,2.204479e-03,4.902219e-06,2.061337e-04
LossLogger.post_iteration,4900,2.408529,4.914362e-04,2.455365e-07,6.345801e-05


In [ ]:
# --- evaluate the results ----
%nbagg
with mx.timer( 'predicting '):
    Yh = model.to('cpu')( X_da.to_tensor( dtype=dtype, device='cpu' ) )
Yh_da = Yh.to_dataarray( Y_da )

fig = figure( figsize=(10, 10 ) )
with mx.timer( 'regressing' ):    
    reg = mx.Regression( autocorr=[1, 13, 26] )
    rms = []
    #for i, (smp, mask) in pbar( enumerate( [('trg', trg_mask.values), ('val', val_mask.values), ('tst',tst_mask.values), ('full',range(len(Y_da))) ] ) ):
    for i, (smp, mask) in pbar( enumerate( [('trg', arange(20000,30000)), ('val', arange(35000,40000)), ('tst',tst_mask.values), ('full',range(len(Y_da))) ] ) ):
        rm = reg.regress( Yh_da[mask], Y_da[mask], W_da[mask] )    
        ax = subplot( 2,2, i+1 )
        ax.plot( rm.pop('pnl').cumsum() )
        ax.set_title( smp, fontsize=15 )
        hr = rm.pop('hitRate')
        rm['sample'] = smp
        rms.append( rm )
disp( pd.DataFrame( rms ).set_index('sample') )

In [ ]:
# run a grid of hyperparameters ...
####################################

# hyperparameters
mx.Config.hps = fr.HPSet( 
  
)
mx.Config.code = ''

# configs = []
# for obs_weight in ['no_weight', 'dollar_weight', 'log_weight']:
#     cfg = fr.Config.clone()
#     cfg.hps = cfg.hps.clone( obs_weight=obs_weight )
#     cfg.freeze()
#     display_html( cfg )
#     configs.append( cfg )
# notebook = fr.Config.run_grid( configs, outfile=fr.Config.uri+'/../cadre_price_index.20200302 - weight_grid_results.ipynb', n_jobs=None )

configs = []
fr.Config.hps.obs_weight = 'log_weight'
for cbsa_factors in [5, 8, 10]:
    cfg = fr.Config.clone()
    cfg.hps = cfg.hps.clone( cbsa_factors=cbsa_factors )
    cfg.freeze()
    #display_html( cfg )
    configs.append( cfg )
notebook = fr.Config.run_grid( configs, outfile=fr.Config.uri+'/../grid_results.ipynb', n_jobs=None )


# fr.Config.hps.obs_weight = 'log_weight'
# fr.Config.hps.cbsa_factors = 8
# configs = []
# for tv in [1, 1e-1, 1e-2, 1e-3]:
#     cfg = fr.Config.clone()
#     cfg.hps = cfg.hps.clone( obs_weight=obs_weight )
#     cfg.freeze()
#     #display_html( cfg )
#     configs.append( cfg )
# notebook = fr.Config.run_grid( configs, outfile=fr.Config.uri+'/../cadre_price_index.20200302 - tv_grid_results.ipynb', n_jobs=None )


